In [ ]:
# Load LSTM network and generate text
import codecs
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
# load ascii text and covert to lowercase
filename = "/home/jvdzwaan/data/tmp/lstm-alice/wonderland.txt"
with codecs.open(filename, 'r', encoding='utf-8') as f:
    raw_text = f.read().strip().lower()
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print "Total Characters: ", n_chars
print "Total Vocab: ", n_vocab
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print "Total Patterns: ", n_patterns
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [ ]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
# load the network weights
filename = "/home/jvdzwaan/data/tmp/lstm-alice/1.9122.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
import copy
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
p = copy.deepcopy(pattern)
print len(pattern)
print len(p)
print "Seed:"
print "\"", ''.join([int_to_char[value] for value in pattern]), "\""

# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    #index = numpy.random.choice(n_vocab, p=prediction[0])
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print "\nDone."
print len(p)

In [ ]:
pattern = copy.deepcopy(p)
print "Seed:"
print "\"", ''.join([int_to_char[value] for value in pattern]), "\""
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    #print prediction
    #print prediction.sum()
    #index = numpy.argmax(prediction)
    index = numpy.random.choice(n_vocab, p=prediction[0])
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print "\nDone."
print len(p)
p

In [ ]:
def select_char(prediction, index_next, top=10):
    """Wanneer kies je voor het gegeven next character (uit de tekst) en wanneer kies je een ander char (op basis van de 
    prediction)? 
    Eerste idee: als index_next niet in de top 10 zit, dan sample je uit prediction, ander houd je index_next
    (of argmax uit prediction)
    """
    #print 'index next character:', index_next
    ind = numpy.argpartition(prediction, -top)[-top:]
    #print ind
    if index_next not in ind:
        #return numpy.argmax(prediction)
        return numpy.random.choice(n_vocab, p=prediction)
    return index_next

for i, j in enumerate(prediction[0]):
    print '{}: {:.4f}'.format(i, j)
select_char(prediction[0], 0)

In [ ]:
begin = [39,
 21,
 28,
 28,
 1,
 39,
 17,
 25,
 36,
 6,
 1,
 17,
 35,
 1,
 35,
 24,
 21,
 1,
 24,
 17,
 20,
 1,
 30,
 31,
 36,
 24,
 25,
 30,
 23,
 1,
 21,
 28,
 35,
 21,
 1,
 36,
 31,
 1,
 20,
 31,
 6,
 1,
 17,
 30,
 20,
 0,
 32,
 21,
 34,
 24,
 17,
 32,
 35,
 1,
 17,
 22,
 36,
 21,
 34,
 1,
 17,
 28,
 28,
 1,
 25,
 36,
 1,
 29,
 25,
 23,
 24,
 36,
 1,
 36,
 21,
 28,
 28,
 1,
 24,
 21,
 34,
 1,
 35,
 31,
 29,
 21,
 36,
 24,
 25,
 30,
 23,
 1,
 39,
 31,
 34,
 36,
 24,
 1,
 24,
 21]
vervolg = u'aring. For some\nminutes it puffed away without speaking, but at last it unfolded its\narms, took the hookah out of its mouth again, and said, ‘So you think\nyou’re changed, do you?’'

In [ ]:
print len(begin)
matches = 0
t = []
print len(vervolg)
for c in vervolg.lower():
    #print c
    x = numpy.reshape(begin, (1, len(begin), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    
    index_next = char_to_int[c]
    result = select_char(prediction[0], index_next)
    
    begin.append(result)
    begin = begin[1:len(begin)]
    
    if int_to_char[result] == c:
        matches += 1
    t.append(int_to_char[result])
print matches
print ''.join(t)
print 
print vervolg

In [ ]:
for i, j in zip(vervolg, ''.join(t)):
    print i, j